In [1]:
import lightfeaturesextract
import load_model as lm
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from sklearn.utils import compute_class_weight
from tensorflow.keras.callbacks import ReduceLROnPlateau
import random
from tensorflow.keras.optimizers import Adam, SGD

ModuleNotFoundError: No module named 'cv2'

In [ ]:
model = lightfeaturesextract.light_featex()

In [ ]:
model.summary()

In [ ]:
spliced, copy_moved, spliced_mask, copy_moved_mask = lm.load_images("../data/CASIA2/Tp/", "../data/CASIA2/gt/")

In [ ]:
data, labels = lm.patch_images(spliced[:100], spliced_mask[:100])

In [ ]:
labels2 = []
for label in labels:
    tp = np.sum(label)/255
    percent = tp*100/(32*32)
    if 10 < percent and percent < 70 :
        labels2.append(1)
    else:
        labels2.append(0)

In [ ]:
data = [rgb.astype('float32')/255. for rgb in data]
labels = labels2

In [ ]:
tmp = list(zip(data, labels))
random.shuffle(tmp)
data, labels = zip(*tmp)

In [ ]:
tt = np.sum(labels)
count = 0
dataf = []
labelsf = []
for k, img in enumerate(data):
    if labels[k] == 0:
        if count <= 4*tt:
            count += 1
            dataf.append(img)
            labelsf.append(labels[k])
    if labels[k] == 1:
        dataf.append(img)
        labelsf.append(labels[k])
data = dataf
labels = labelsf

In [ ]:
classWeight = compute_class_weight('balanced', np.unique(labels), labels) 
classWeight = dict(enumerate(classWeight))
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=1e-6)

In [ ]:
train_img, val_img, train_labels, val_labels = train_test_split(data, labels, test_size=0.1, random_state=42)

In [ ]:
history = model.fit(train_img, train_labels, epochs=20, batch_size = 32,
                    validation_data=(val_img, val_labels))